<a href="https://colab.research.google.com/github/arfild/dw_matrix/blob/master/Day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install hyperopt

In [19]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin


%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd'/content/drive/My Drive/Colab Notebook/matrix/matrix_tree/dw_matrix_road_sign/data'

/content/drive/My Drive/Colab Notebook/matrix/matrix_tree/dw_matrix_road_sign/data


In [0]:
train = pd.read_pickle('train.p')
test = pd.read_pickle('test.p')

[X_train, y_train] = train['features'], train['labels']
[X_test, y_test] = test['features'], test['labels']

In [0]:

df=pd.read_csv('signnames.csv')
labels = df.to_dict()['b']




In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),  # rozmiar paczki
      epochs=params_fit.get('epochs', 5),            # liczba iteracji
      verbose=params_fit.get('verbose', 1),          # stan
      validation_data=params_fit.get('validation_data', (X_train, y_train)),  # do krzywych uczenia się
      callbacks=[tensorboard_callback]               # do wizualizacji
  )
  return model

In [0]:

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),
    
    Dense(num_classes, activation='softmax')
  ])

In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.6209 - accuracy: 0.2876 - val_loss: 1.0883 - val_accuracy: 0.6750
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 0.9600 - accuracy: 0.6956 - val_loss: 0.2981 - val_accuracy: 0.8985
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.4765 - accuracy: 0.8500 - val_loss: 0.1167 - val_accuracy: 0.9675
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3067 - accuracy: 0.9029 - val_loss: 0.0639 - val_accuracy: 0.9814
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2375 - accuracy: 0.9259 - val_loss: 0.0479 - val_accuracy: 0.9858


0.9566893424036281

In [11]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.1919 - accuracy: 0.9567


[0.191934734582901, 0.9566893577575684]

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_1']),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_2']),
    
    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_3']),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_1']),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_2']),
    
    Dense(num_classes, activation='softmax')
  ])

In [33]:

model_trained.evaluate(X_test, y_test)


138/138 [==============================] - 0s 3ms/step - loss: 2.9406 - accuracy: 0.2224


[2.940563678741455, 0.22244897484779358]

In [0]:
def func_obj(params):
  get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),  # rozmiar paczki
      epochs=7,            # liczba iteracji
      verbose=0,          # stan
  )

  score = model.evaluate(X_test, y_test,verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [35]:
space = {
      'batch_size': hp.quniform('batch_size', 50, 200, 20),
      'dropout_cnn_block_1': hp.uniform('dropout_cnn_block_1', 0.3, 0.5),
      'dropout_cnn_block_2': hp.uniform('dropout_cnn_block_2', 0.3, 0.5),
      'dropout_cnn_block_3': hp.uniform('dropout_cnn_block_3', 0.3, 0.5),
      'dropout_dense_block_1': hp.uniform('dropout_dense_block_1', 0.3, 0.7),
      'dropout_dense_block_2': hp.uniform('dropout_dense_block_2', 0.3, 0.7),
}

best = fmin( func_obj, space, tpe.suggest, 30, Trials())

{'batch_size': 60.0, 'dropout_cnn_block_1': 0.3850993453909563, 'dropout_cnn_block_2': 0.481949877198552, 'dropout_cnn_block_3': 0.36901017440842376, 'dropout_dense_block_1': 0.41831051893630966, 'dropout_dense_block_2': 0.5387693369955172}
accuracy=0.054421767592430115
{'batch_size': 60.0, 'dropout_cnn_block_1': 0.3605534699040267, 'dropout_cnn_block_2': 0.41147157620381386, 'dropout_cnn_block_3': 0.3632425170370843, 'dropout_dense_block_1': 0.5948156824388473, 'dropout_dense_block_2': 0.5197652262295871}
accuracy=0.054421767592430115
{'batch_size': 100.0, 'dropout_cnn_block_1': 0.48921440183977355, 'dropout_cnn_block_2': 0.3507046267038851, 'dropout_cnn_block_3': 0.31842895815157163, 'dropout_dense_block_1': 0.3895681058240207, 'dropout_dense_block_2': 0.42569979402964897}
accuracy=0.054421767592430115
{'batch_size': 180.0, 'dropout_cnn_block_1': 0.3206669070933507, 'dropout_cnn_block_2': 0.48341284982762084, 'dropout_cnn_block_3': 0.44099674023417124, 'dropout_dense_block_1': 0.6664